## Udacity CS212 Design of computer programs

### Lesson 1

Phases
                        1                    2                   3                  4
Vague understanding --------- > Problem -----------> Spec ---------------> Code ---------->
                    understand           specify             design                test

**Writing a poker program**

**max** function

In [2]:
l = [3, 4, -5, 0]
print( max(l), max(l, key=abs) )

4 -5


**key=abs** applies built-in abs function to every element and then find max element
Nb: max returns the orginal value **-5** and not the key applied value **5**

In [3]:
def poker(hands):
    "Return the best hand: poker([hand,...]) => hand"

In [4]:
def poker(hands):
    "Return the best hand: poker([hand,...]) => hand"
    return max(hands, key=hand_rank)

def hand_rank(hand):
    return None # we will be changing this later.

**Testing**

Before moving on to define hand_rank which will be the complex function in this program, we'll be better off thinking about how the functions will be used and writing some testcases.

It's important that each part of the specification gets turned into a piece of code that implements it and a test that tests it.

In [5]:
# Test for the poker function
def poker(hands):
    "Return the best hand: poker([hand,...]) => hand"
    return max(hands, key=hand_rank)

def test():
    "Test cases for the functions in poker program"
    sf = "6C 7C 8C 9C TC".split() # => ['6C', '7C', '8C', '9C', 'TC']
    fk = "9D 9H 9S 9C 7D".split() 
    fh = "TD TC TH 7C 7D".split()
    
    # test1 - straight flush > four-of-a-kind > full house
    assert poker([sf, fk, fh]) == sf
    
    # test2 - four of a kind (fk) vs. full house (fh) returns fk
    assert poker([fk, fh]) == fk
    
    # test3 - full house (fh) vs. full house (fh) returns fh
    assert poker([fh, fh]) == fh    
    
    return 'Tests passed'

One important principle of testing is to test extreme values eg: test3

What if the list passed into **poker** function has only 0, 1, 100 items?
    
    (Poker isn't played alone, so there's no point passing a list of single element into the function. But as the specification doesn't rule out chances of input list size being 1, we'll have to consider that too)
    
    The specification doesn't mention what to do if an empty list is passed, but it does specify that the **poker** function returns a **hand**. So we could **return a None** or **raise an error**

In [7]:
# Tests for 1 hand and 100 hands
def test():
    "Test cases for the functions in poker program"
    sf = "6C 7C 8C 9C TC".split() 
    fk = "9D 9H 9S 9C 7D".split() 
    fh = "TD TC TH 7C 7D".split()
    
    # test4 - 1 hand passed as input
    assert poker([fh]) == fh
    
    list_of_hands = []
    len_of_hands = 100
    hands_dict = {'sf': sf, 'fk': fk, 'fh':fh}
    
    check_rank = 0 # check presence of hands in list, 0-fh 1-fk 2-sf
    for _ in range(len_of_hands):
        hand = random.choice(hands_dict.keys()) # select a random value from 'sf', 'fk' and 'fh'
        list_of_hands.append(hands_dict[hand])  # append the corresponding dict value to the list
        
        if(check_rank < 2):
            if(hand == 'fh'):
                check_rank = 2
            elif(hand == 'fk'):
                check_rank = 1
    
    
    assert len(list_of_hands) == len_of_hands # assert the list does have the required no. of hands
    
    # test5 - n hands passed to the poker function
    if(check_rank == 2):
        assert poker(list_of_hands) == sf # returns sf if sf in list
    elif(check_rank == 1):
        assert poker(list_of_hands) == fk # returns fk if fk, fh only in list
    else:
        assert poker(list_of_hands) == fh # returns fh if only fh in list
    
    '''
    Another version of test4
        assert poker([sf] +  99*[fh]]) == sf
    '''
    return 'tests pass'

### The hand_rank function

There are 9 ranks from straight flush to high card, 0-8

In [5]:
def hand_rank(hand):
    "Return a value indicating the rank of the hand"
    ranks = card_ranks(hand) # returns a list of card numbers in sorted order eg: 10 9 8 7 6
    
    if straight(ranks) and flush(hand):
        '''
        if straight can be determined by just looking at the sequence of numbers of the cards(suite isn't required)
        To check for a flush we need to know if all the cards are of the same suit => check hand
        '''
        return 8
    elif kind(4, ranks):
        return 7
    ...

This approach provides the correct answer for most of the cases. But consider this case:
Person1: 10, 10, 9, 8, 5
Person2: 9, 9, 6, 5, 2

Here hand_rank() function returns the value 1(pair), but we know that pair of 10s are ranked higher than pair of 9s

We need to use more complex alternatives
<img src="Screenshots/hand-rank-possibility-quiz.jpg">

Here all three will work, but the best solution will be to use **tuples** as we can easily get the contents. With **int** and **real**, we'll have to use further processing to extract the numbers

In [1]:
(7, 9, 5) > (7, 3, 2)

True

Ranking cards
<img src="Screenshots/Screenshot (715).png">
<img src="Screenshots/Screenshot (716).png">

<p style="text-align:center;font-weight:bolder">(8, 11) > (7, 14, 2) Lexicographic ordering</p>

In [6]:
def hand_rank(hand):
    "Return a value indicating the rank of the hand"
    ranks = card_ranks(hand) # returns a list of card numbers in sorted order eg: 10 9 8 7 6 
    
    if straight(ranks) and flush(hand):
        return (8, max(ranks)) # 2 3 4 5 6 (8, 6)  6 7 8 9 T (8, 10)
    elif kind(4, ranks):
        return (7, kind(4, ranks), kind(1, ranks)) # 9 9 9 9 3 (7, 9, 3)
        # kind(n, ranks) returns the value that repeats n times kind(4, [10, 10, 10, 10, 2]) => 10
    ...
    

In [3]:
def test():
    "Test cases for the functions in poker program"
    sf = "6C 7C 8C 9C TC".split() # Straight Flush
    fk = "9D 9H 9S 9C 7D".split() # Four of a Kind
    fh = "TD TC TH 7C 7D".split() # Full House
    assert poker([sf, fk, fh]) == sf
    assert poker([fk, fh]) == fk
    assert poker([fh, fh]) == fh
    assert poker([sf]) == sf
    assert poker([sf] + 99*[fh]) == sf
 
    assert hand_rank(sf) == (8, 10)
    assert hand_rank(fk) == (7, 9, 7)
    assert hand_rank(fh) == (6, 10, 7)

In [10]:
def hand_rank(hand):
    ranks = card_ranks(hand)
    
    if straight(ranks) and flush(hand):            # straight flush
        return (8, max(ranks))
    elif kind(4, ranks):                           # 4 of a kind
        return (7, kind(4, ranks), kind(1, ranks))
    elif kind(3, ranks) and kind(2, ranks):        # full house
        return (6, kind(3, ranks), kind(2, ranks))
    elif flush(hand):                              # flush
        return (5, ranks)
    elif straight(ranks):                          # straight
        return (4, max(ranks))
    elif kind(3, ranks):                           # 3 of a kind
        return (3, kind(3, ranks), ranks)
    elif two_pair(ranks):                          # 2 pair
        return (2, two_pair(2, ranks), ranks)
    elif kind(2, ranks):                           # kind
        return (1, kind(2, ranks), ranks)
    else:                                          # high card
        return (0, ranks)

In [9]:
def test():
    "Test cases for the functions in poker program"
    sf = "6C 7C 8C 9C TC".split() # Straight Flush
    fk = "9D 9H 9S 9C 7D".split() # Four of a Kind
    fh = "TD TC TH 7C 7D".split() # Full House
    assert card_ranks(sf) == [10, 9, 8, 7, 6]
    assert card_ranks(fk) == [9, 9, 9, 9, 7]
    assert card_ranks(fh) == [10, 10, 10, 7, 7]
    
    assert poker([sf, fk, fh]) == sf
    assert poker([fk, fh]) == fk
    assert poker([fh, fh]) == fh
    assert poker([sf]) == sf
    assert poker([sf] + 99*[fh]) == sf
    
    assert hand_rank(sf) == (8, 10)
    assert hand_rank(fk) == (7, 9, 7)
    assert hand_rank(fh) == (6, 10, 7)
    return 'tests pass'

#### card_ranks functions

Input: a hand of cards

Output: sorted list of card ranks

In [12]:
def card_ranks(cards):
    "Return a list of the ranks, sorted with higher first."
    ranks = [r for r,s in cards]
    ranks.sort(reverse=True)
    return ranks

print(card_ranks(['AC', '3D', '4S', 'KH'])) #should output [14, 13, 4, 3]

['K', 'A', '4', '3']


But **A** is of higher rank than **K** and we need to print out numbers

In [16]:
def card_ranks(cards):
    "Return a list of the ranks, sorted with higher first."
    ranks_dict = {}
    for i in range(2, 9):
        ranks_dict[str(i)] = i
    i = 10
    for j in ['T', 'J', 'Q', 'K', 'A']:
        ranks_dict[j] = i
        i += 1
    
    ranks = [ranks_dict[r] for r,s in cards] # index function returns the index of the element in the list
    ranks.sort(reverse=True)
    return ranks

print(card_ranks(['AC', '3D', '4S', 'KH'])) #should output [14, 13, 4, 3]

[14, 13, 4, 3]


**OR**

In [17]:
def card_ranks(cards):
    "Return a list of the ranks, sorted with higher first."    
    ranks = ['--23456789TJQKA'.index(r) for r,s in cards] # index function returns the index of the element in the list
    ranks.sort(reverse=True)
    return ranks

print(card_ranks(['AC', '3D', '4S', 'KH'])) #should output [14, 13, 4, 3]

[14, 13, 4, 3]


**It's better to first write tests before writing the functions**

In [20]:
def test():
    "Test cases for the functions in poker program."
    sf = "6C 7C 8C 9C TC".split()
    fk = "9D 9H 9S 9C 7D".split()
    fh = "TD TC TH 7C 7D".split()
    assert straight([9, 8, 7, 6, 5]) == True
    assert straight([9, 8, 8, 6, 5]) == False
    assert flush(sf) == True
    assert flush(fk) == False
    return 'tests pass'

def straight(ranks):
    "Return True if the ordered ranks form a 5-card straight."
    for i in range(len(ranks)-1):
        if (ranks[i] - ranks[i+1]) != 1:
            return False
    return True
'''
def straight(ranks):
    "Return True if the ordered ranks form a 5-card straight."
    return (max(ranks) - min(ranks) == 4) and (len(set(ranks)) == 5)

'''
def flush(hand):
    "Return True if all the cards have the same suit."
    # Your code here.
    suits = set([s for r, s in hand])
    return len(suits) == 1

print(test())

tests pass


In [26]:
def test():
    "Test cases for the functions in poker program."
    sf = "6C 7C 8C 9C TC".split() # Straight Flush
    fk = "9D 9H 9S 9C 7D".split() # Four of a Kind
    fh = "TD TC TH 7C 7D".split() # Full House
    tp = "5S 5D 9H 9C 6S".split() # Two pairs
    fkranks = card_ranks(fk)
    tpranks = card_ranks(tp)

    assert kind(4, fkranks) == 9
    assert kind(3, fkranks) == None
    assert kind(2, fkranks) == None
    assert kind(1, fkranks) == 7
    return 'tests pass'

def kind(n, ranks):
    """Return the first rank that this hand has exactly n of.
    Return None if there is no n-of-a-kind in the hand."""
    for r in ranks:
        if ranks.count(r) == n:
            return r
    return None
print(test())

tests pass
